In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
# chapter1 = pd.read_csv("C://Users//mynam//TariffBot//data//section1_data//htsdata_Chapter1.csv")

In [ ]:
# chapter1.shape

(126, 9)

In [5]:
# chapter1.sample(10)

In [ ]:
# chapter2 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter2.csv")

In [ ]:
# chapter3 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter3.csv")
# chapter4 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter4.csv")
# # chapter5 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter5.csv")

In [6]:
# chapter2.head()

In [7]:
# chapter3.sample(6)

In [8]:
# chapter4.head()

In [9]:
# chapter5.head()

In [10]:
# merged_df = pd.concat([chapter1, chapter2, chapter3, chapter4, chapter5], ignore_index=True)

In [ ]:
# merged_df.columns

Index(['HTS Number', 'Indent', 'Description', 'Unit of Quantity',
       'General Rate of Duty', 'Special Rate of Duty', 'Column 2 Rate of Duty',
       'Quota Quantity', 'Additional Duties'],
      dtype='object')

In [ ]:
# merged_df.to_csv("section1.csv",index=False)

In [40]:
df = pd.read_csv("C:\\Users\\mynam\\TariffBot\\HTS-Tariff-Agent\\data\\downloads\\section1.csv")

In [50]:
df['HTS Number']

0                0101
1                 NaN
2          0101.21.00
3       0101.21.00.10
4       0101.21.00.20
            ...      
1664    0511.99.40.24
1665    0511.99.40.28
1666    0511.99.40.30
1667    0511.99.40.40
1668    0511.99.40.70
Name: HTS Number, Length: 1669, dtype: object

In [41]:
df['HTS Number'].isnull().sum()

np.int64(287)

In [42]:
df_cleaned = df.dropna(subset=['HTS Number']).copy()

In [43]:
df_cleaned['HTS Number'].max()

'0511.99.40.70'

In [44]:
# Ensures consistent string operations and preserves leading zeros
df_cleaned['HTS Number'] = df_cleaned['HTS Number'].astype(str)

# Trim whitespace
df_cleaned['HTS Number'] = df_cleaned['HTS Number'].str.strip()

In [47]:
def standardize_hts(hts_code):

    # Remove all dots and then re-add them based on length
    numeric_part = hts_code.replace('.', '')

    # Example: If you want 10 digits always (like XXXXXX.XXXX) or c
    # This example assumes a structure like 4.2.2.2 (10 digits, 3 dots)
    if len(numeric_part) == 4: # e.g., '0101' -> '0101.00.00.00'
        return f"{numeric_part}.00.00.00"
    elif len(numeric_part) == 6:
        return f"{numeric_part[:4]}.{numeric_part[4:6]}.00.00"
    elif len(numeric_part) == 8: # e.g., '01012100' or '0101.21.00'
        return f"{numeric_part[:4]}.{numeric_part[4:6]}.{numeric_part[6:8]}.00"
    elif len(numeric_part) == 10: # e.g., '0101210010' or '0101.21.00.10'
        return f"{numeric_part[:4]}.{numeric_part[4:6]}.{numeric_part[6:8]}.{numeric_part[8:10]}"
    # Add more conditions for other lengths you might encounter
    else:
        # If it doesn't fit a standard pattern, you might return it as is,
        # or mark it for review, or set to NaN
        print(f"Warning: HTS '{hts_code}' (numeric: '{numeric_part}') does not fit standard format. Returning as is.")
        return hts_code # Or pd.NA for invalid ones

df_cleaned['HTS Number'] = df_cleaned['HTS Number'].apply(standardize_hts)

In [48]:
df_cleaned.sample(15)

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties,HTS Number Cleaned
721,0304.59.00.09,5,Other,"[""kg""]",NaN,NaN,NaN,NaN,NaN,0304.59.00.09
1441,0406.30.34.00,3,Described in additional U.S. note 19 to this c...,"[""kg"",""kg cmsc""]",20%,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",35%,NaN,NaN,0406.30.34.00
963,0306.11.00.00,2,Rock lobster and other sea crawfish (Palinurus...,NaN,Free,NaN,Free,NaN,NaN,0306.11.00.00
1573,0406.90.94.00,6,Other,"[""kg"",""kg cmsc""]",$1.128/kg,"Free (BH,CL,JO,MA,OM,P,PE,SG) 7.5¢/kg (KR) See...",$1.328/kg,NaN,NaN,0406.90.94.00
115,0106.33.00.00,2,Ostriches; emus (Dromaius novaehollandiae),"[""No.""]",1.8%,"Free (A,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,PE...",20%,NaN,NaN,0106.33.00.00
1117,0307.72.00.60,3,Other,"[""kg""]",NaN,NaN,NaN,NaN,NaN,0307.72.00.60
617,0303.89.00.10,3,Cusk,"[""kg""]",NaN,NaN,NaN,NaN,NaN,0303.89.00.10
248,0203.21.00.00,2,Carcasses and half-carcasses,"[""kg""]",Free,NaN,5.5¢/kg,NaN,NaN,0203.21.00.00
1535,0406.90.44.00,3,Described in general note 15 of the tariff sch...,"[""kg""]",6.4%,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",35%,NaN,NaN,0406.90.44.00
284,0206.00.00.00,0,"Edible offal of bovine animals, swine, sheep, ...",NaN,NaN,NaN,NaN,NaN,NaN,0206.00.00.00


In [55]:
df[df["HTS Number"].astype(str).str.startswith("0101.21.00")]

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties
2,0101.21.00,2,Purebred breeding animals,NaN,Free,NaN,Free,NaN,NaN
3,0101.21.00.10,3,Males,"[""No.""]",NaN,NaN,NaN,NaN,NaN
4,0101.21.00.20,3,Females,"[""No.""]",NaN,NaN,NaN,NaN,NaN


In [14]:
import re

# Load HTS data CSV
# df = pd.read_csv("htsdata.csv")

# Input values
product_cost = 10000.0  # FOB cost
freight = 500.0
insurance = 100.0
unit_weight = 1000  # in kg
quantity = 10       # number of units

# Compute CIF
cif_value = product_cost + freight + insurance

# Duty parser
def parse_duty_advanced(duty_str, unit_weight=None, quantity=None):
    """
    Parses and returns a duty rate as a decimal fraction (e.g., 0.05 for 5%),
    or a calculated specific duty based on unit_weight or quantity.
    """
    if pd.isna(duty_str) or duty_str.strip() == "":
        return 0.0

    duty_str = duty_str.strip().lower()

    if "free" in duty_str:
        return 0.0

    # Percentage duty (e.g., '5%')
    match = re.search(r"([\d.]+)\s*%", duty_str)
    if match:
        return float(match.group(1)) / 100

    # Weight-based duty (e.g., '2.5¢/kg')
    match = re.search(r"([\d.]+)\s*¢/kg", duty_str)
    if match and unit_weight is not None:
        cents_per_kg = float(match.group(1))
        return (cents_per_kg * unit_weight) / (100 * cif_value)  # convert to % of CIF

    # Unit-based duty (e.g., '$1.00/unit')
    match = re.search(r"\$([\d.]+)/unit", duty_str)
    if match and quantity is not None:
        dollars_per_unit = float(match.group(1))
        return (dollars_per_unit * quantity) / cif_value

    return 0.0

# Create working DataFrame
duty_df = df[["HTS Number", "Description", "General Rate of Duty", 
              "Special Rate of Duty", "Column 2 Rate of Duty"]].copy()

# Apply CIF and input data
duty_df["CIF Value"] = cif_value
duty_df["Product Cost"] = product_cost
duty_df["Freight"] = freight
duty_df["Insurance"] = insurance

# Apply duty calculations
for col in ["General Rate of Duty", "Special Rate of Duty", "Column 2 Rate of Duty"]:
    parsed_col = f"{col} Parsed (%)"
    amount_col = f"{col} Duty Amount"
    duty_df[parsed_col] = duty_df[col].apply(lambda x: parse_duty_advanced(x, unit_weight, quantity))
    duty_df[amount_col] = duty_df[parsed_col] * cif_value

# Filter results with any applicable duty
duty_df_filtered = duty_df[
    (duty_df["General Rate of Duty Duty Amount"] > 0) |
    (duty_df["Special Rate of Duty Duty Amount"] > 0) |
    (duty_df["Column 2 Rate of Duty Duty Amount"] > 0)
]

# Save results
duty_df_filtered.to_csv("landed_cost_duties.csv", index=False)


In [2]:
landed_cost = pd.read_csv("landed_cost_duties.csv")

In [11]:
landed_cost['HTS Number']

0         0101.29.00
1      0101.30.00.00
2      0101.90.40.00
3         0102.29.20
4         0102.29.40
           ...      
513    0510.00.20.00
514       0511.10.00
515       0511.99.30
516    0511.99.36.00
517       0511.99.40
Name: HTS Number, Length: 518, dtype: object

In [17]:
landed_cost[landed_cost["HTS Number"].astype(str).str.startswith("0101.21")]

,HTS Number,Description,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,CIF Value,Product Cost,Freight,Insurance,General Rate of Duty Parsed (%),General Rate of Duty Duty Amount,Special Rate of Duty Parsed (%),Special Rate of Duty Duty Amount,Column 2 Rate of Duty Parsed (%),Column 2 Rate of Duty Duty Amount


In [49]:
data_Store_path = "C:\\Users\\mynam\\TariffBot\\HTS-Tariff-Agent\\data\\downloads\\section1.csv"
df_cleaned.to_csv(data_Store_path,index=False)